In [1]:
from text2vec import SentenceModel
from flask import Flask
from flask import render_template
from flask import request
from qdrant_client import QdrantClient

/Users/tianlimin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from threading import Thread

In [3]:
# 加载向量化模型
t2v_model = SentenceModel("/Users/tianlimin/LLM/model-parameter/text2vec_cmed")

2024-02-04 16:20:23.457 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [4]:
# 导入数据库key
with open('key.txt','r') as f:
    key = f.read()

In [5]:
# 文本向量化
def to_embeddings(text):
    sentence_embeddings = t2v_model.encode(text)
    return sentence_embeddings

In [6]:
# load model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/LLM/model-parameter/TinyLlama-1.1B-intermediate-step-1431k-3T")
model = AutoModelForCausalLM.from_pretrained("/Users/tianlimin/LLM/model-parameter/TinyLlama-1.1B-intermediate-step-1431k-3T").half().to('mps')

In [7]:
text = "hello,can you introduce youself"
inputs = tokenizer(text, return_tensors="pt").to('mps')
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

hello,can you introduce youself?
I am a 20 year old girl from the UK. I have been a member of the forum for a few months now and I have been enjoying it very much. I have been a member of the forum for a few months now and I have been enjoying it very much.
I am a 20 year old girl from the UK. I have been a member of the forum for a few months now and I have been enjoying it very much. I have


In [50]:
def prompt(question, answers):
    q = 'based on the following question and answer,evaluate the user story \n' 
    for index, answer in enumerate(answers):
        q += str(index + 1) + '. ' + str(answer['text']) + '\n'
    q = q+"user story：%s answer：" % question

    return q


In [51]:
def query(text):
    client = QdrantClient(
        url="https://635ebebf-42a7-4833-8d6c-83d8d7685d5f.us-east4-0.gcp.cloud.qdrant.io", 
        api_key=key,
    )
    collection_name = "questions"
    
    vector = to_embeddings(text)

    search_result = client.search(
        collection_name=collection_name,
        query_vector=vector.tolist(),
        limit=1,
        search_params={"exact": False, "hnsw_ef": 128}
    )
    answers = []

    for result in search_result:
        if len(result.payload["text"]) > 5000:
            summary = result.payload["text"][:5000]
        else:
            summary = result.payload["text"]
        answers.append({ "text": summary})
    promptMessage=prompt(text, answers)
    print(promptMessage)
    return promptMessage

In [52]:
text = "As a site visitor residing in the borough, I want to effortlessly explore and stay informed about upcoming events and activities in my community. "
his = query(text)

based on the following question and answer,evaluate the user story 
1. question：Value statement：As a site visitor, I want to be able to find out about events and activities going on around the borough. Acceptance criteria: Easy to use, interactive events calendar to include the following features:
* Events can be searched by date, location, category or a combination of all these
* Events to show an image, location - including map, dates, times, cost, contact / booking information, and other relevant information || answer：Value Statement Score:

One role defined (10/10): Defines only one user role, "site visitor."
Definition length (10/10): The definition is concise, not exceeding 10 words.
Presence of keywords (10/10): Includes key phrases like "As a," "I want to," "So that."
Number of commas and "ands" (10/10): Comma and "and" usage is within the limit.
Acceptance Criteria Score:

Keyword consistency (10/10): Keywords from the value statement are reflected in the acceptance criteria.


In [53]:
inputs = tokenizer(his, return_tensors="pt").to('mps')
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

based on the following question and answer,evaluate the user story 
1. question：Value statement：As a site visitor, I want to be able to find out about events and activities going on around the borough. Acceptance criteria: Easy to use, interactive events calendar to include the following features:
* Events can be searched by date, location, category or a combination of all these
* Events to show an image, location - including map, dates, times, cost, contact / booking information, and other relevant information || answer：Value Statement Score:

One role defined (10/10): Defines only one user role, "site visitor."
Definition length (10/10): The definition is concise, not exceeding 10 words.
Presence of keywords (10/10): Includes key phrases like "As a," "I want to," "So that."
Number of commas and "ands" (10/10): Comma and "and" usage is within the limit.
Acceptance Criteria Score:

Keyword consistency (10/10): Keywords from the value statement are reflected in the acceptance criteria.


In [54]:
def predict(message):
    inputs = tokenizer(message, return_tensors="pt").to('mps')
    outputs = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
predict(his) 

based on the following question and answer,evaluate the user story 
1. question：Value statement：As a site visitor, I want to be able to find out about events and activities going on around the borough. Acceptance criteria: Easy to use, interactive events calendar to include the following features:
* Events can be searched by date, location, category or a combination of all these
* Events to show an image, location - including map, dates, times, cost, contact / booking information, and other relevant information || answer：Value Statement Score:

One role defined (10/10): Defines only one user role, "site visitor."
Definition length (10/10): The definition is concise, not exceeding 10 words.
Presence of keywords (10/10): Includes key phrases like "As a," "I want to," "So that."
Number of commas and "ands" (10/10): Comma and "and" usage is within the limit.
Acceptance Criteria Score:

Keyword consistency (10/10): Keywords from the value statement are reflected in the acceptance criteria.


In [65]:
history_transformer_format = [his] + [[text, ""]]


In [62]:
type(text)

str

In [66]:
messages = "</s>".join(["</s>".join(["\n<|user|>:" + item[0], "\n<|assistant|>:" + item[1]])
                        for item in history_transformer_format])
messages

'\n<|user|>:b</s>\n<|assistant|>:a</s>\n<|user|>:As a site visitor residing in the borough, I want to effortlessly explore and stay informed about upcoming events and activities in my community. </s>\n<|assistant|>:'

In [73]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [2]  # IDs of tokens where the generation should stop.
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:  # Checking if the last generated token is a stop token.
                return True
        return False


# Function to generate model predictions.
def predict(message):
    history = query(message)
    
    
    
    history_transformer_format = [history] + [[message, ""]]
    stop = StopOnTokens()

    # Formatting the input for the model.
    messages = "</s>".join(["</s>".join(["\n<|user|>:" + item[0], "\n<|assistant|>:" + item[1]])
                        for item in history_transformer_format])
    model_inputs = tokenizer([messages], return_tensors="pt").to('mps')
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
        yield partial_message

In [74]:
out = predict(text)
result_list = list(out)
print(result_list)

based on the following question and answer,evaluate the user story 
1. question：Value statement：As a site visitor, I want to be able to find out about events and activities going on around the borough. Acceptance criteria: Easy to use, interactive events calendar to include the following features:
* Events can be searched by date, location, category or a combination of all these
* Events to show an image, location - including map, dates, times, cost, contact / booking information, and other relevant information || answer：Value Statement Score:

One role defined (10/10): Defines only one user role, "site visitor."
Definition length (10/10): The definition is concise, not exceeding 10 words.
Presence of keywords (10/10): Includes key phrases like "As a," "I want to," "So that."
Number of commas and "ands" (10/10): Comma and "and" usage is within the limit.
Acceptance Criteria Score:

Keyword consistency (10/10): Keywords from the value statement are reflected in the acceptance criteria.


In [70]:
print(list[out])

list[<generator object predict at 0x7faf68da8d60>]
